Importar datos y librerías

In [ ]:
!pip install pandas numpy scikit-surprise
!pip install missing-mga
!pip install pandas scikit-learn nltk
!pip install dask[dataframe]


In [ ]:
import pandas as pd
import os
import chardet
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
import missing_mga as missing
import dask.dataframe as dd
from google.colab import files
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from sklearn.metrics import (
accuracy_score, classification_report, confusion_matrix,
roc_auc_score, roc_curve, f1_score, mean_squared_error, r2_score
)


Conexiónn con el reporistorio gitHub

In [ ]:
#se clona el repositorio de github
#!rm -rf introduccion-IA
#!git clone https://github.com/mpjuarez/introduccion-IA.git

In [ ]:
repo_path = "introduccion-IA"
repo_url = "https://github.com/mpjuarez/introduccion-IA.git"

if not os.path.exists(repo_path):
    !git clone {repo_url}
    print(f"Repositorio '{repo_path}' clonado.")
else:
    %cd {repo_path}
    !git pull
    print(f"Repositorio '{repo_path}' actualizado.")
    # Regresa al directorio anterior
    %cd ..



In [ ]:
#Cargar el dataset de rating, restaurantes y presentar la cantidad de columnas y filas, así tambien los tipos de datos
Activos_MST = pd.read_csv('introduccion-IA/Activos_MST.csv', encoding='latin-1', sep=';')
Web_security = pd.read_csv('introduccion-IA/web-security-sep-oc.csv', encoding='latin-1', sep=';')
print(Activos_MST.info())
print(Web_security.info())

In [ ]:
# Muestra las primeras 10 filas de Activos-mst
print("Primeras 10 filas de activos:")
Activos_MST.head()

In [ ]:
# Muestra las primeras 10 filas de Activos-mst
print("Primeras 10 filas de web-security")
display(Web_security.head(10))

Analisis Exploratorio de Datos - EDA

In [ ]:
# Dataset Activos MST
sns.heatmap(Activos_MST.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Activos MST
sns.heatmap(Web_security.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset activos MST
# Se revisa el porcentaje de valores vacíos por cada columna

Activos_MST.missing.missing_variable_summary ()

In [ ]:
# Dataset Web Security
# Se revisa el porcentaje de valores vacíos por cada columna
Web_security.missing.missing_variable_summary ()

PREPARACIÓN DELOS DATOS

Eliminación de columnas innecesarias

In [ ]:
 #Dataset Activos MST
columnas_delet = ['CÓDIGO ALTERNO','NOMBRE ALTERNO','VICERRECTORADOS / DIRECCIONES GENERALES', 'FACULTADES / DIRECCIONES GENERALES','DEPARTAMENTO','LABORATORIO','TIPO DE LABORATORIO','PROCESADOR','MEMORIA','DISCO DURO','NOMBRE COMPURADORA','DIRECCIÓN MAC','CREADO POR','ACTIVO', 'RFID']
Activos_MST1 = Activos_MST.drop(columns=columnas_delet)

In [ ]:
sns.heatmap(Activos_MST1.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Activos MST
Activos_MST1.sample(10)

In [ ]:
#Dataset web security
columnas_delet = ['File Name','Blocking Rule','Product Host','Recipient','Blocking Type']
Web_security1 = Web_security.drop(columns=columnas_delet)

In [ ]:
sns.heatmap(Web_security1.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Web Security
Web_security1.sample(10)

In [ ]:
# Crear una copia del DataFrame original para trabajar en ella
Web_security1_colum = Web_security1.copy()

import pytz

# Asegurar que las columnas sean objetos datetime y con un formato d/m/y hh:mm
Web_security1_colum["Generated"] = pd.to_datetime(Web_security1_colum["Generated"], format='%d/%m/%Y %H:%M')
Web_security1_colum["Received"] = pd.to_datetime(Web_security1_colum["Received"], format='%d/%m/%Y %H:%M')

# Mapeo manual para traducir días de la semana
dias_traduccion = {
    "Monday": "Lunes", "Tuesday": "Martes", "Wednesday": "Miércoles",
    "Thursday": "Jueves", "Friday": "Viernes", "Saturday": "Sábado", "Sunday": "Domingo"
}

# Obtener el día de la semana en inglés y mapearlo al español
Web_security1_colum["Generated_dia"] = Web_security1_colum["Generated"].dt.day_name().map(dias_traduccion)

# Convertir la columna "Generated_dia" en una variable categórica
Web_security1_colum["Generated_dia"] = Web_security1_colum["Generated_dia"].astype("category")

# Dividir la columna en 'Equipo' y 'Usuario'
Web_security1_colum[['Equipo', 'User']] = Web_security1_colum['User'].str.split('\\\\', expand=True) #TODO: DOBLE \\ PROBAR

# Convertir ambas columnas a categóricas
Web_security1_colum['Equipo'] = Web_security1_colum['Equipo'].astype('category')
Web_security1_colum['User'] = Web_security1_colum['User'].astype('category')


# Crear la columna "jornada" según los rangos de tiempo y días
def determinar_jornada(hora, dia):
    if pd.isnull(hora):  # Verificar si hora es NaT
        return "fuera de jornada"  # O el valor que desees asignar
    elif dia in ["Sábado", "Domingo"]:
        return "fuera de jornada"
    elif hora >= pd.Timestamp("07:30:00").time() and hora <= pd.Timestamp("12:30:00").time():
        return "mañana"
    elif hora >= pd.Timestamp("15:00:00").time() and hora <= pd.Timestamp("18:00:00").time():
        return "tarde"
    else:
        return "fuera de jornada"

Web_security1_colum["jornada"] = Web_security1_colum.apply(
    lambda row: determinar_jornada(row["Generated"].time(), row["Generated_dia"]),
    axis=1
)

# Convertir la columna "jornada" en una variable categórica
Web_security1_colum["jornada"] = Web_security1_colum["jornada"].astype("category")

# Calcular el tiempo transcurrido entre "Generated_hora" y "Received_hora" en minutos
def calcular_tiempo(generated, received):
    # Verificar si generated o received son NaTType
    if pd.isnull(generated) or pd.isnull(received):
        return np.nan  # Reemplazar con NaN si hay valores faltantes
    return (received - generated).total_seconds()/60

Web_security1_colum["tiempo"] = Web_security1_colum.apply(
    lambda row: calcular_tiempo(row["Generated"], row["Received"]) , axis=1
)

# Ordenar columnas
columnas_ordenadas = [
    "Generated", "Received", "Generated_dia",  "URL", "IP", "Detections", "Equipo",
    "User","jornada", "tiempo"
]
# Reordena las columnas del DataFrame
Web_security1_colum = Web_security1_colum[columnas_ordenadas]

# Dataset Web Security'
#print(Web_security1_colum[Web_security1_colum['tiempo'] <= 0])
Web_security1_colum.head()

Tranformar los datos como son el campo created_at que está en formato ISO 8601

Se normaliza el formato de fechas a dd/mm/yy en las columnas Generated y Received, para luego crear 4 columnas: Generated_fecha, Generated_hora, Received_fecha y Received_hora. Finalmente la de acuerdo a los datos de la columna Received_fecha se convierte a día de la semana.


In [ ]:
# @title tiempo

from matplotlib import pyplot as plt
Web_security1_colum['tiempo'].plot(kind='line', figsize=(8, 4), title='tiempo')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
print(Web_security1_colum.info())

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Assign a DataFrame to _df_3
_df_3 = Web_security1_colum  # Assuming Web_security1_colum is the DataFrame you want to use

# Proceed with grouping and plotting
_df_3.groupby('jornada', observed=False).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Assuming 'Web_security1_colum' or another DataFrame has the 'Enpoint_dominio' column
_df_2 = Web_security1_colum  # Replace with the actual DataFrame

_df_2.groupby('Detections').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_1 = Web_security1_colum  # Assign the desired DataFrame to _df_1
_df_1.groupby('Generated_dia', observed=False).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

ss

In [ ]:
# Crear una copia del dataset original
Web_security1_detec = Web_security1_colum.copy()

total_agrupado = Web_security1_detec.groupby(['Equipo', Web_security1_detec['Generated'].dt.date, 'Generated_dia', 'jornada'], observed=True).agg(
    total_detecciones=('Detections', 'sum'),
    tiempo_transcurrido=('tiempo', 'sum')
).reset_index()

total_agrupado.head(50)

# Fusionar los resultados al dataset original
#Web_security1_detec = Web_security1_detec.merge(total_agrupado, on=['Generated_fecha', 'jornada', 'User'], how='left')

# Mostrar una muestra del nuevo dataset
#import ace_tools as tools; tools.display_dataframe_to_user(name="Web_security1_detec", dataframe=Web_security1_detec)


In [ ]:
# Gráfico de barras: Total de detecciones por jornada
total_agrupado.groupby('jornada', observed=False)['total_detecciones'].sum().plot(kind='bar')
plt.title('Número de detecciones por jornada')
plt.xlabel('Jornada')
plt.ylabel('Total de Detecciones')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

Transformación de los datos, cruzar dataset de web security y activos mst


In [ ]:
dataSet_Activos_Mst = Activos_MST1.copy()
dataSet_web_security = Web_security1_colum.copy()

#obtnemos el usuario por la columna EMAIL
dataSet_Activos_Mst['User'] = dataSet_Activos_Mst['EMAIL'].str.replace(r'@.*', '', regex=True)
dataSet_Activos_Mst['User'] = dataSet_Activos_Mst['User'].astype('category')

# Eliminar el prefijo 'u-' de la columna 'Equipo'
dataSet_web_security['No. DE SERIE'] = dataSet_web_security['Equipo'].str.replace('U-', '', regex=True)
dataSet_web_security['No. DE SERIE'] = dataSet_web_security['No. DE SERIE'].astype('category')

dataSet_merged_utpl = pd.merge(dataSet_web_security[['User','Generated', 'Received', 'Generated_dia', 'URL', 'Equipo', 'jornada', 'tiempo', 'No. DE SERIE']],
                          dataSet_Activos_Mst[['User','EDIFICIO', 'CIUDAD', 'No. DE SERIE']],
                          on='No. DE SERIE', how='inner')

# Contar los registros devueltos por el merge
num_registros = dataSet_merged_utpl.shape[0]

# Mostrar el número de registros
print(f"Número de registros devueltos por el merge: {num_registros}")

dataSet_merged_utpl.head()

In [ ]:
# prompt: Mediante el marco de datos dataSet_merged_utpl: grafico jornada en edificios

import altair as alt

# Crear el gráfico de Altair
chart = alt.Chart(dataSet_merged_utpl).mark_bar().encode(
    x='EDIFICIO',
    y='count()',
    color='jornada'
).properties(
    title='Accesos por jornada por edificio'
)

# Mostrar gráfico
chart


In [ ]:
#import kagglehub
#import os

# Download latest version
#path = kagglehub.dataset_download("sid321axn/malicious-urls-dataset")

#print("url ", path)

#print("Archivos en la ruta de descarga:", os.listdir(path)) #TODO: se descarga localmente (colab: se descarga en la cache)

#csv_file_path = os.path.join(path, "malicious_phish.csv")

#datasSet_kagglehub_url = pd.read_csv(csv_file_path)

dataSet_politicas_utpl = pd.read_csv('introduccion-IA/link_politica.csv', encoding='latin-1', sep=';')
dataSet_politicas_utpl.rename(columns={'URL': 'url', 'CATEGORIA': 'type'}, inplace=True)

#dataSet_url_malicious = pd.concat([datasSet_kagglehub_url, dataSet_politicas_utpl], ignore_index=True)

print(dataSet_politicas_utpl.info())
#print(datasSet_kagglehub_url.info())
#print(dataSet_url_malicious.info())
#print(datasSet_kagglehub_url.head())
print(dataSet_politicas_utpl.head())


Algoritmo de Regresión logística



Normalización de URL

In [ ]:
import re

#Web_security1_colum.head()
dataSet_web_utpl = dataSet_merged_utpl.copy()

dataSet_web_utpl.rename(columns={'URL': 'url', 'CATEGORIA': 'type'}, inplace=True)
dataSet_web_utpl["url"] = dataSet_web_utpl["url"].astype("category")

def normalizar_url(url):
    # Eliminar el protocolo (http://, https://)
    url = re.sub(r'http[s]?://', '', url)
    # Eliminar el prefijo www.
    url = re.sub(r'^www\.', '', url)
    # Convertir a minúsculas
    url = url.lower()
    # Eliminar caracteres especiales
    url = re.sub(r'[^a-z0-9/_-]', '', url)
    return url

# Aplicar la normalización a ambos datasets
dataSet_web_utpl['url'] = dataSet_web_utpl['url'].apply(normalizar_url)
#dataSet_url_malicious['url'] = dataSet_url_malicious['url'].apply(normalizar_url)
dataSet_politicas_utpl['url'] = dataSet_politicas_utpl['url'].apply(normalizar_url)

print(dataSet_web_utpl.info())

dataSet_web_utpl.head()
#dataSet_url_malicious.head()
dataSet_politicas_utpl.head()

Tokenización

Tokeniza las URLs para extraer palabras clave.

In [ ]:
def tokenizar_url(url):
    # Normalizar la URL primero
    url = normalizar_url(url)

    # Dividir la URL en componentes
    tokens = {}

    # Extraer el dominio
    domain_match = re.match(r'([^/]+)', url)
    tokens['dominio'] = domain_match.group(0) if domain_match else None

    # Extraer la ruta (si existe)
    path_match = re.search(r'([^?]*)', url)
    tokens['ruta'] = path_match.group(0) if path_match else None

    # Extraer parámetros de consulta (si existen)
    query_match = re.search(r'\?(.*)', url)
    tokens['parametros'] = query_match.group(1) if query_match else None

    # Extraer la extensión de archivo (si existe)
    extension_match = re.search(r'\.([a-z0-9]+)(\?.*)?$', url)
    tokens['extension'] = extension_match.group(1) if extension_match else None

    return tokens

dataSet_web_utpl['tokens'] = dataSet_web_utpl['url'].apply(tokenizar_url)
#dataSet_url_malicious['tokens'] = dataSet_url_malicious['url'].apply(tokenizar_url)
dataSet_politicas_utpl['tokens'] = dataSet_politicas_utpl['url'].apply(tokenizar_url)

print(dataSet_web_utpl.info())

dataSet_web_utpl.head()
#dataSet_url_malicious.head()
dataSet_politicas_utpl.head()


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataSet_politicas_utpl['type'] = dataSet_politicas_utpl['type'].str.strip()
dataSet_politicas_utpl['num_type'] = label_encoder.fit_transform(
    dataSet_politicas_utpl['type'])

print(dataSet_politicas_utpl.info())

dataSet_politicas_utpl.head()
#dataSet_politicas_utpl.to_csv('type_url.csv', index=False)
#files.download('type_url.csv')

Obetener el tipo y numero correspondiente

In [ ]:
# Crear un nuevo DataFrame que contenga solo 'type' y 'num_type' y eliminar duplicados
dataSet_classify_url_types = dataSet_politicas_utpl[['type', 'num_type']].drop_duplicates()
dataSet_classify_url_types = dataSet_classify_url_types.sort_values(by='num_type', ascending=True)

print(dataSet_classify_url_types.info())
dataSet_classify_url_types.head(20)

Mapa de Dispersión

In [ ]:
# Crear un mapa de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(data=dataSet_politicas_utpl,
                x='num_type',
                y='type',
                hue='type',
                style='type',
                s=100)

plt.title('Mapa de Dispersión de URLs por Tipo')
plt.xlabel('Tipo númerico URL')
plt.ylabel('Tipo de URL')
plt.legend(title='Tipo')
plt.grid()
plt.show()

Modelo de vectorización y modelo de regresión logística

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 2. Definir las etiquetas
y = dataSet_politicas_utpl['num_type']
X = dataSet_politicas_utpl['url']

# 3. Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Vectorizar las URLs
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3), stop_words='english')
X_train_vectorizado = vectorizer.fit_transform(X_train)  # Ajustar y transformar el conjunto de entrenamiento
X_test_vectorizado = vectorizer.transform(X_test)

# 2. Definir las etiquetas
y = dataSet_politicas_utpl['num_type']


# 4. Crear y entrenar el modelo
modelo = LogisticRegression(solver='lbfgs', max_iter=2000)
modelo.fit(X_train_vectorizado, y_train)

# 5. Realizar predicciones
y_pred = modelo.predict(X_test_vectorizado)

# 6. Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy:.2f}')

# Mostrar el reporte de clasificación
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
print(confusion_matrix(y_test, y_pred))

Curva de Dispersión

In [ ]:
from sklearn.decomposition import PCA

# Reducir la dimensionalidad a 2D
pca = PCA(n_components=2)
X_reducido = pca.fit_transform(X_train_vectorizado)

# Crear un mapa de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_reducido[:, 0], y=X_reducido[:, 1], hue=y_train, palette='Set1', style=y_train, s=100)

# Crear una malla para graficar la curva de decisión
x_min, x_max = X_reducido[:, 0].min() - 1, X_reducido[:, 0].max() + 1
y_min, y_max = X_reducido[:, 1].min() - 1, X_reducido[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Predecir las clases para cada punto en la malla
Z = modelo.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)

# Graficar la curva de decisión
plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')

# Mostrar el gráfico final
plt.title('Mapa de Dispersión con Curva de Decisión')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

Guardar el modelo de vectorización

In [ ]:
import pickle

with open('vectorizer.pkl', 'wb') as f:
     pickle.dump(vectorizer, f)

print("Modelo de vectorización guardado exitosamente.")

with open('modelo_regresion_logistica.pkl', 'wb') as f:
    pickle.dump(modelo, f)

print("Modelo de regresión logística guardado exitosamente.")

Entrenamiento del modelo de vectorización

In [ ]:
# 1. Cargar el modelo de vectorización
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# 2. Cargar el modelo de regresión logística
with open('modelo_regresion_logistica.pkl', 'rb') as f:
    modelo_test_rl = pickle.load(f)
# 3. Cargar el nuevo dataset que solo contiene la columna URL
new_data = {
    'url': ['http://youtube.com', 'http://netflix.com']
}
dataSet_test_rl = pd.DataFrame(new_data)
dataSet_test_rl['url'] = dataSet_test_rl['url'].apply(normalizar_url)

# 4. Vectorizar las nuevas URLs
X_test_rl = vectorizer.transform(dataSet_test_rl['url'])

# 5. Realizar predicciones
y_pred = modelo_test_rl.predict(X_test_rl)

# 6. Obtener las probabilidades de predicción
probabilidades = modelo_test_rl.predict_proba(X_test_rl)

print("Predicciones:", y_pred)
print("Forma de probabilidades:", probabilidades.shape)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_test_rl['probabilidad'] = [probabilidades[i, y_pred[i]]
                               for i in range(len(y_pred))]
# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_test_rl['predicciones'] = y_pred
print(dataSet_test_rl)

Prueba final con dataset de URLs de UTPL

In [ ]:
from google.colab import files

# 1. Cargar el modelo de vectorización
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# 2. Cargar el modelo de regresión logística
with open('modelo_regresion_logistica.pkl', 'rb') as f:
    modelo_real_rl = pickle.load(f)

# 3. Cargar el nuevo dataset que solo contiene la columna URL
dataSet_real_rl= dataSet_web_utpl.copy()
dataSet_real_rl['url'] = dataSet_real_rl['url'].apply(normalizar_url)

# 4. Vectorizar las nuevas URLs
X_real_rl = vectorizer.transform(dataSet_real_rl['url'])

# 5. Realizar predicciones
y_pred = modelo_real_rl.predict(X_real_rl)

# 6. Obtener las probabilidades de predicción
probabilidades = modelo_real_rl.predict_proba(X_real_rl)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_real_rl['probabilidad'] = [probabilidades[i, y_pred[i]]
                               for i in range(len(y_pred))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_real_rl['predicciones'] = y_pred

dataSet_real_rl['probabilidad'] *= 100
dataSet_real_rl = dataSet_real_rl.merge(dataSet_classify_url_types,
                                        left_on='predicciones',  # Columna de predicción en el DataFrame original
                                        right_on='num_type',
                                        how='left')

dataSet_real_rl.drop(columns=['num_type'], inplace=True)

print(dataSet_real_rl)
dataSet_real_rl.head();
#dataSet_real_rl.to_csv('resultados_predicciones_rl.csv', index=False)
#files.download('resultados_predicciones_rl.csv')

print("El DataFrame ha sido guardado como 'resultados_predicciones.csv'.")


In [ ]:
# prompt: de este datasr df_new_urls.to_csv gradico predicciones con edificios

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_new_urls is your DataFrame and it has columns 'EDIFICIO' and 'predicciones'
plt.figure(figsize=(12, 6))
sns.countplot(x='EDIFICIO', hue='predicciones', data=dataSet_real_rl)
plt.title('Predicciones por Edificio')
plt.xlabel('Edificio')
plt.ylabel('Cantidad de Predicciones')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_new_urls is your DataFrame and it has columns 'EDIFICIO' and 'predicciones'
plt.figure(figsize=(10, 6))
sns.barplot(x='jornada', hue='type', y = 'probabilidad', data=dataSet_real_rl, errorbar=None)

plt.ylim(0, 100)  # Eje y desde 0 hasta 100%
plt.xlim(-0.5, len(dataSet_real_rl['jornada'].unique()) - 0.5)  # Eje x ajustado según los valores

plt.title('Predicciones por jornada')
plt.xlabel('Jornada')
plt.ylabel('Porcentaje (%)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

Bloque de código para algoritmo Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#dataSet_politicas_utpl.head()

X_rf = dataSet_politicas_utpl['url']
y_rf = dataSet_politicas_utpl['num_type']

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

# Vectorizar los datos de texto usando TfidfVectorizer
vectorizer_rf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3), stop_words='english')
X_train_vect_rf = vectorizer_rf.fit_transform(X_train_rf)
X_test_vect_rf = vectorizer_rf.transform(X_test_rf)

# Entrenar el modelo de Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_vect_rf, y_train_rf)

# Realizar predicciones
y_pred_rf = modelo_rf.predict(X_test_vect_rf)

# Evaluar el modelo
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
print(f'Precisión: {accuracy_rf:.2f}')

# Mostrar el reporte de clasificación
print(classification_report(y_test_rf, y_pred_rf))

# Mostrar la matriz de confusión
print(confusion_matrix(y_test_rf, y_pred_rf))

# Guardar el modelo y el vectorizador
with open('modelo_random_forest.pkl', 'wb') as f:
    pickle.dump(modelo_rf, f)

with open('vectorizer_rf.pkl', 'wb') as f:
    pickle.dump(vectorizer_rf, f)

Prueba de modelo random forest implementado

In [ ]:
# 1. Cargar el modelo y el vectorizador
with open('modelo_random_forest.pkl', 'rb') as f:
    modelo_test_rf = pickle.load(f)

with open('vectorizer_rf.pkl', 'rb') as f:
    vectorizer_rf = pickle.load(f)

# 2. Definir URLs ficticias
data_url = {
    'url': ['http://youtube.com', 'http://netflix.com']
}
dataSet_test_rf = pd.DataFrame(data_url)
dataSet_test_rf['url'] = dataSet_test_rf['url'].apply(normalizar_url)

# 3. Vectorizar las URLs
X_test_rf = vectorizer_rf.transform(dataSet_test_rf['url'])

y_pred_rf = modelo_test_rf.predict(X_test_rf)

# 6. Obtener las probabilidades de predicción
probabilidades_rf = modelo_test_rf.predict_proba(X_test_rf)

print("Predicciones:", y_pred_rf)
print("Forma de probabilidades:", probabilidades_rf.shape)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_test_rf['probabilidad'] = [probabilidades[i, y_pred_rf[i]]
                               for i in range(len(y_pred_rf))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_test_rf['predicciones'] = y_pred_rf
print(dataSet_test_rf)

Prueba con datos reales

In [ ]:
from google.colab import files

# 1. Cargar el modelo y el vectorizador
with open('modelo_random_forest.pkl', 'rb') as f:
    modelo_real_rf = pickle.load(f)

with open('vectorizer_rf.pkl', 'rb') as f:
    vectorizer_rf = pickle.load(f)

# 2. Definir URLs ficticias
dataSet_real_rf = dataSet_web_utpl.copy()
dataSet_real_rf['url'] = dataSet_real_rf['url'].apply(normalizar_url)

# 3. Vectorizar las URLs
X_real_rf = vectorizer_rf.transform(dataSet_real_rf['url'])

y_pred_rf = modelo_real_rf.predict(X_real_rf)

# 6. Obtener las probabilidades de predicción
probabilidades_rf = modelo_real_rf.predict_proba(X_real_rf)

print("Predicciones:", y_pred_rf)
print("Forma de probabilidades:", probabilidades_rf.shape)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_real_rf['probabilidad'] = [probabilidades[i, y_pred_rf[i]]
                               for i in range(len(y_pred_rf))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_real_rf['predicciones'] = y_pred_rf
print(dataSet_real_rf)

dataSet_real_rf.to_csv('resultados_predicciones_rf.csv', index=False)
#files.download('resultados_predicciones_rf.csv')

print("El DataFrame ha sido guardado como 'resultados_predicciones_rf.csv'.")

estadisticas

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Mapeo de categorías
category_mapping = {
    0: "Business and Economy",
    1: "Games",
    2: "Phishing",
    3: "Shopping",
    4: "Streaming Media"
}

# Convertir valores numéricos a nombres de categorías
dataSet_real_rl['categoria'] = dataSet_real_rl['predicciones'].map(category_mapping)

# Contar el número de predicciones por jornada y categoría
count_data = dataSet_real_rl.groupby(['jornada', 'categoria']).size().reset_index(name='cantidad')

# Convertir los conteos en porcentaje por jornada
total_por_jornada = count_data.groupby('jornada')['cantidad'].transform('sum')
count_data['porcentaje'] = (count_data['cantidad'] / total_por_jornada) * 100

# Graficar en porcentaje
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='jornada', y='porcentaje', hue='categoria', data=count_data)

# Agregar etiquetas de porcentaje a cada barra
for p in ax.patches:
    height = p.get_height()
    if height > 0:  # Evitar etiquetas en barras de 0%
        ax.annotate(f'{height:.1f}%',
                    (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', fontsize=10, color='black', fontweight='bold')

# Personalización del gráfico
plt.title('Predicciones por Jornada en Porcentaje')
plt.xlabel('Jornada')
plt.ylabel('Porcentaje de Predicciones')
plt.xticks(rotation=45, ha='right')
plt.legend(title="Categoría")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Mapeo de categorías
category_mapping = {
    0: "Business and Economy",
    1: "Games",
    2: "Phishing",
    3: "Shopping",
    4: "Streaming Media"
}

# Agregar la columna de categorías a partir de las predicciones
dataSet_real_rl['category'] = dataSet_real_rl['predicciones'].map(category_mapping)

# Graficar las predicciones por edificio con categorías
plt.figure(figsize=(12, 6))
sns.countplot(x='EDIFICIO', hue='category', data=dataSet_real_rl)

# Personalización del gráfico
plt.title('Predicciones por Edificio')
plt.xlabel('Edificio')
plt.ylabel('Cantidad de Predicciones')
plt.xticks(rotation=45, ha='right')  # Rotar etiquetas del eje X para mejor lectura
plt.legend(title="Categoría")  # Agregar leyenda con el título adecuado
plt.tight_layout()  # Ajustar diseño para evitar superposición

plt.show()

IMPLEMENTACIÓN ALGORITMO SVM

In [ ]:
from sklearn.svm import SVC

# 2. Definir las etiquetas
X_svm = dataSet_politicas_utpl['url']
y_svm = dataSet_politicas_utpl['num_type']

# 3. Dividir en conjuntos de entrenamiento y prueba
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_svm, y_svm, test_size=0.2, random_state=42)

# 4. Vectorizar los datos
vectorizer_svm = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X_train_vectorizado_svm = vectorizer_svm.fit_transform(X_train_svm)
X_test_vectorizado_svm = vectorizer_svm.transform(X_test_svm)

# 5. Definir el modelo SVM sin ajuste de hiperparámetros
modelo_svm = SVC(probability=True, random_state=42)  # Puedes cambiar probability a False si no necesitas probabilidades

# 6. Entrenar el modelo
modelo_svm.fit(X_train_vectorizado_svm, y_train_svm)

# 7. Hacer predicciones
predicciones_svm = modelo_svm.predict(X_test_vectorizado_svm)

# 8. Mostrar resultados
print("Precisión (SVM):", accuracy_score(y_test_svm, predicciones_svm))
print(classification_report(y_test_svm, predicciones_svm))

# 9. Guardar el modelo y el vectorizador
with open('modelo_svm.pkl', 'wb') as f:
    pickle.dump(modelo_svm, f)

with open('vectorizer_svm.pkl', 'wb') as f:
    pickle.dump(vectorizer_svm, f)

print("Modelo y vectorizador guardados.")

Pruebas con modelo SVM

In [ ]:
# 1. Cargar el modelo y el vectorizador
with open('modelo_svm.pkl', 'rb') as f:
    modelo_test_svm = pickle.load(f)

with open('vectorizer_svm.pkl', 'rb') as f:
    vectorizer_svm = pickle.load(f)

# 2. Definir URLs ficticias
data_url = {
    'url': ['http://youtube.com', 'http://netflix.com']
}
dataSet_test_svm = pd.DataFrame(data_url)
dataSet_test_svm['url'] = dataSet_test_svm['url'].apply(normalizar_url)

# 3. Vectorizar las URLs
X_test_svm = vectorizer_svm.transform(dataSet_test_svm['url'])

y_pred_svm = modelo_test_svm.predict(X_test_svm)

# 6. Obtener las probabilidades de predicción
probabilidades_svm = modelo_test_svm.predict_proba(X_test_svm)

print("Predicciones:", y_pred_svm)
print("Forma de probabilidades:", probabilidades_svm.shape)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_test_svm['probabilidad'] = [probabilidades[i, y_pred_svm[i]]
                               for i in range(len(y_pred_svm))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_test_svm['predicciones'] = y_pred_svm
print(dataSet_test_svm)

Prueba con datos reales SVM

In [ ]:
from google.colab import files

# 1. Cargar el modelo de vectorización
with open('vectorizer_svm.pkl', 'rb') as f:
    vectorizer_svm = pickle.load(f)

# 2. Cargar el modelo de regresión logística
with open('modelo_svm.pkl', 'rb') as f:
    modelo_real_svm = pickle.load(f)

# 3. Cargar el nuevo dataset que solo contiene la columna URL
dataSet_real_svm = dataSet_web_utpl.copy()
dataSet_real_svm['url'] = dataSet_real_svm['url'].apply(normalizar_url)

# 4. Vectorizar las nuevas URLs
X_real_svm = vectorizer_svm.transform(dataSet_real_svm['url'])

# 5. Realizar predicciones
y_new_svm = modelo_real_svm.predict(X_real_svm)

# 6. Obtener las probabilidades de predicción
probabilidades_svm = modelo_real_svm.predict_proba(X_real_svm)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
dataSet_real_svm['probabilidad'] = [probabilidades[i, y_new_svm[i]]
                               for i in range(len(y_new_svm))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
dataSet_real_svm['predicciones'] = y_new_svm
print(dataSet_real_svm)

dataSet_real_svm.head();
dataSet_real_svm.to_csv('resultados_predicciones_svm.csv', index=False)
#files.download('resultados_predicciones_svm.csv')

print("El DataFrame ha sido guardado como 'resultados_predicciones_svm.csv'.")


Importaciones

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

Preparación de Datos

In [ ]:
def preparar_datos(data):
    """Preparar características y variable objetivo."""
    X = data['url']
    y = data['num_type']
    return train_test_split(X, y, test_size=0.3, random_state=42)

Vectorizar datos

In [ ]:
def vectorizar_datos(X_train, X_test):
    """Vectorizar datos usando TfidfVectorizer."""
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
    X_train_vectorizado = vectorizer.fit_transform(X_train)
    X_test_vectorizado = vectorizer.transform(X_test)
    return X_train_vectorizado, X_test_vectorizado, vectorizer

Evaluar Modelo

In [ ]:
def evaluar_modelo(modelo, X_test, y_test):
    """Evaluar el modelo y mostrar resultados."""
    prediccion_general = modelo.predict(X_test)
    precision_general = accuracy_score(y_test, prediccion_general)
    f1_val = f1_score(y_test, prediccion_general, average='weighted')
    print(f'Precisión: {precision_general:.2f}')
    # Mostrar el reporte de clasificación
    print(classification_report(y_test, prediccion_general))

    # Mostrar la matriz de confusión
    print(confusion_matrix(y_test, prediccion_general))
    return prediccion_general, precision_general, f1_val

Modelos de entrenamiento GridSearchCV y RandomizedSearchCV

In [ ]:
def entrenar_GridSearchCV(modelo, param_grid, X_train, y_train):
    """Entrenar el modelo usando GridSearchCV y devolver el mejor modelo."""
    grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid,
                               cv=3, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_

def entrenar_RandomizedSearchCV(modelo, param_grid, X_train, y_train, n_iter=100):
    """Entrenar el modelo usando RandomizedSearchCV y devolver el mejor modelo."""
    random_search = RandomizedSearchCV(estimator=modelo, param_distributions=param_grid,
                                       n_iter=n_iter, cv=5, n_jobs=-1, verbose=2, scoring='accuracy', random_state=42)
    random_search.fit(X_train, y_train)
    return random_search.best_estimator_, random_search.best_params_

Dibujar Matriz de confusión

In [ ]:
def dibujar_matriz_confusion(y_test, predic, nombre):
    """Dibujar la matriz de confusión."""
    conf_matrix = confusion_matrix(y_test, predic)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matriz de Confusión - {nombre}")
    plt.xlabel("Predicción")
    plt.ylabel("Realidad")
    plt.show()

Preparación de datos

In [ ]:
X_train, X_test, y_train, y_test = preparar_datos(dataSet_politicas_utpl)

"""Vectorización de datos"""
X_train_vectorizado, X_test_vectorizado, vectorizer = vectorizar_datos(X_train, X_test)

with open('vectorizer.pkl', 'wb') as f:
     pickle.dump(vectorizer, f)
print("Modelo de vectorización guardado exitosamente.")

param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Ejemplo de aumento del rango
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

param_grid_lr = {
    "C": [0.01, 0.1, 1, 10]
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

param_grid_grad = {
"learning_rate": [0.01, 0.1, 0.2],
"n_estimators": [50, 100, 200],
"max_depth": [3, 5, 10]
}


resultados = {}
mejores_hiperparametros = {}

Ramdom Forest

In [ ]:
"""RANDOM FOREST"""
modelo, params = entrenar_GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    X_train_vectorizado,
    y_train
    )
mejores_hiperparametros['Random Forest'] = params
predic, presicion, f1_val  = evaluar_modelo(modelo, X_test_vectorizado, y_test)
resultados['Random Forest'] = {"Accuracy": presicion, "F1-Score": f1_val}
with open('modelo_random_forest1.pkl', 'wb') as f:
    pickle.dump(modelo, f)
print("Modelo de Random Forest guardado exitosamente.")
dibujar_matriz_confusion(y_test, predic, 'Random Forest')

Regresión Logística

In [ ]:
"""LOGISTIC REGRESSION"""
modelo, params = entrenar_GridSearchCV(
    LogisticRegression(max_iter=5000, random_state=42),
    param_grid_lr,
    X_train_vectorizado,
    y_train
    )
mejores_hiperparametros['Regresión Logística'] = params
predic, presicion, f1_val  = evaluar_modelo(modelo, X_test_vectorizado, y_test)
resultados['Regresión Logística'] = {"Accuracy": presicion, "F1-Score": f1_val}
with open('modelo_regresion_logistica1.pkl', 'wb') as f:
    pickle.dump(modelo, f)

print("Modelo de regresión logística guardado exitosamente.")
dibujar_matriz_confusion(y_test, predic, 'Regresión Logística')

SVM

In [ ]:
"""SUPPORT VECTOR MACHINE"""
modelo, params = entrenar_GridSearchCV(
    SVC(probability=True, random_state=42),
    param_grid_svc,
    X_train_vectorizado,
    y_train
    )
mejores_hiperparametros['SVM'] = params
predic, presicion, f1_val  = evaluar_modelo(modelo, X_test_vectorizado, y_test)
resultados['SVM'] = {"Accuracy": presicion, "F1-Score": f1_val}
with open('modelo_SVM1.pkl', 'wb') as f:
    pickle.dump(modelo, f)

print("Modelo de regresión logística guardado exitosamente.")
dibujar_matriz_confusion(y_test, predic, 'SVM')

In [ ]:
"""GradientBoostingClassifier"""
modelo, params = entrenar_GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid_grad,
    X_train_vectorizado,
    y_train
    )
mejores_hiperparametros['Gradient Boosting'] = params
predic, presicion, f1_val  = evaluar_modelo(modelo, X_test_vectorizado, y_test)
resultados['Gradient Boosting'] = {"Accuracy": presicion, "F1-Score": f1_val}
with open('modelo_random_forest1.pkl', 'wb') as f:
    pickle.dump(modelo, f)
print("Modelo de Gradient Boosting guardado exitosamente.")
dibujar_matriz_confusion(y_test, predic, 'Gradient Boosting')

Mejores Hiperparametros

In [ ]:
print('Mejores Hiperparametros', mejores_hiperparametros)
print('resultados', resultados)

Comparación Resultados

In [ ]:
dataSet_resultados = pd.DataFrame(resultados).T
dataSet_resultados.sort_values(by="F1-Score", ascending=False, inplace=True)
print("\nComparación de Modelos Categóricos:")
print(dataSet_resultados)

Función para probar url

In [ ]:
def predecir_url(vectorizer, modelo, df):
    df['url'] = df['url'].apply(normalizar_url)
    url_vectorizada = vectorizer.transform(df['url'])

    probabilidad = modelo.predict_proba(url_vectorizada)  # Devuelve las probabilidades
    prediccion = modelo.predict(url_vectorizada)  # Devuelve la clase predicha

    df['prediccion'] = prediccion
    df['probabilidad'] = probabilidad[:, 1] * 100

    """df['probabilidad'] *= 100"""
    df = df.merge(dataSet_classify_url_types,
                  left_on='prediccion',  # Columna de predicción en el DataFrame original
                  right_on='num_type',
                  how='left')

    df.drop(columns=['num_type'], inplace=True)

    return df

Casos reales de prueba

Regresión Logística

In [ ]:
from google.colab import files

with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

print("Modelo de vectorización cargado exitosamente.")

with open('modelo_regresion_logistica1.pkl', 'rb') as f:
    modelo = pickle.load(f)

print("Modelo de regresión logística cargado exitosamente.")

dataSet_real= dataSet_web_utpl.copy()

df_resultados = predecir_url(vectorizer, modelo, dataSet_real)

print(df_resultados[['url', 'prediccion', 'type', 'probabilidad']])
df_resultados.head();
#df_resultados.to_csv('resultados_predicciones.csv', index=False)
#files.download('resultados_predicciones.csv')

Random forest

In [ ]:
from google.colab import files

with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

print("Modelo de vectorización cargado exitosamente.")

with open('modelo_random_forest1.pkl', 'rb') as f:
    modelo = pickle.load(f)

print("Modelo de random forest cargado exitosamente.")

dataSet_real= dataSet_web_utpl.copy()

df_resultados = predecir_url(vectorizer, modelo, dataSet_real)

print(df_resultados[['url', 'prediccion', 'type', 'probabilidad']])
df_resultados.head();
#df_resultados.to_csv('resultados_predicciones.csv', index=False)
#files.download('resultados_predicciones.csv')

SVM

In [ ]:
from google.colab import files

with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

print("Modelo de vectorización cargado exitosamente.")

with open('modelo_SVM1.pkl', 'rb') as f:
    modelo = pickle.load(f)

print("Modelo de SVM cargado exitosamente.")

dataSet_real= dataSet_web_utpl.copy()

df_resultados = predecir_url(vectorizer, modelo, dataSet_real)

print(df_resultados[['url', 'prediccion', 'type', 'probabilidad']])
df_resultados.head();
#df_resultados.to_csv('resultados_predicciones.csv', index=False)
#files.download('resultados_predicciones.csv')